# MOSTLY AI vs. SDV Comparison - Sequential Scenario  <a href="https://colab.research.google.com/github/mostly-ai/mostlyai/blob/main/docs/tutorials/sdv-comparison/sequential-scenario/sequential-scenario.ipynb" target="_blank"><img src="https://img.shields.io/badge/Open%20in-Colab-blue?logo=google-colab" alt="Run on Colab"></a>

This notebook provides a comprehensive comparison between two leading synthetic data generation platforms:
- **SDV (Synthetic Data Vault)** - Business Source License
- **MOSTLY AI Synthetic Data SDK** - Apache 2.0 License - Open Source

In this comparison, we are going to walk through the synthesis of a relational two-table structure with sequential values using the [Berka dataset](https://github.com/mostly-ai/public-demo-data/tree/dev/berka/data).

## Comparison Methodology

1. **Data Preparation**: Load, inspect, and preprocess the multi-table dataset
2. **Data Splitting**: Create train/test splits
3. **Model Training**: Train both SDV and MOSTLY AI generators on the training data
4. **Synthetic Data Generation**: Generate synthetic datasets using both platforms
5. **Performance Analysis**: Compare training time, generation speed, and data quality

## Key Challenges in Two-Table Synthesis

- **Sequential Dependencies**: Preserving temporal patterns in transaction data
- **Data Quality**: Ensuring synthetic data maintains statistical properties and business logic

In [ ]:
# Install SDK in CLIENT mode
!uv pip install -U mostlyai sdv graphviz
# Or install in LOCAL mode
!uv pip install -U 'mostlyai[local]' sdv graphviz
# Note: Restart kernel session after installation!

## 1. Data Loading and Initial Exploration

First, let's load our multi-table dataset and examine its structure to understand:
- Table schemas and data types
- Data quality and completeness
- Business logic and constraints


In [ ]:
import pandas as pd

base_url = "https://github.com/mostly-ai/public-demo-data/raw/dev/berka/data/"
originals = {
    "account": pd.read_csv(base_url + "account.csv.gz", low_memory=False),
    "transaction": pd.read_csv(base_url + "trans.csv.gz", low_memory=False),
}

# Drop unnecessary columns from transaction table
originals["transaction"].drop(columns=["bank", "account"], inplace=True)

# Convert date columns to datetime
originals["account"]["date"] = pd.to_datetime(originals["account"]["date"])
originals["transaction"]["date"] = pd.to_datetime(originals["transaction"]["date"])

# Display samples
for k in originals:
    print("===", k, "===")
    display(originals[k].sample(n=3))

## 2. Strategic Data Splitting for Multi-Table Scenarios

When dealing with related tables like accounts and transactions, data splitting becomes more complex than simple random sampling due to the links between data tables. 

In order to make coherent assessments of data quality, we need to create meaningful train and test cohorts.

**Key Considerations:**
- **Business Logic:** Accounts and transactions can only exist in the training set if their associated client is also in the training set.
- **Data Leakage Prevention:** Avoid information bleeding between train/test sets.

**Our Approach:**
1. **Split accounts first (80/20 train/test)**: We split the `account` table using an 80/20 ratio. This ensures customer-related information is kept together per split.
   
2. **Assign related tables based on account membership:**  
   - **Transactions:** All linked via `account_id`. These tables follow the same assignment logic.


In [ ]:
from sklearn.model_selection import train_test_split

print("✂️ Performing strategic multi-table data splitting based on accounts...")

# Step 1: Split accounts using 80/20 ratio
accounts_train, accounts_test = train_test_split(originals["account"], test_size=0.2, random_state=42)

print("🏦 Account split:")
print(
    f"   - Training set: {len(accounts_train):,} accounts ({len(accounts_train) / len(originals['account']) * 100:.1f}%)"
)
print(f"   - Test set: {len(accounts_test):,} accounts ({len(accounts_test) / len(originals['account']) * 100:.1f}%)")

# Step 2: Create account ID sets for lookup
train_account_ids = set(accounts_train["account_id"])
test_account_ids = set(accounts_test["account_id"])

In [ ]:
print("🔄 Assigning transactions based on account split...")

# Split transactions linked to training and test accounts
transactions_train = originals["transaction"][originals["transaction"]["account_id"].isin(train_account_ids)].copy()
transactions_test = originals["transaction"][originals["transaction"]["account_id"].isin(test_account_ids)].copy()

print("✅ Splitting complete!")
print(f"   - Training accounts: {len(accounts_train):,}")
print(f"   - Training transactions: {len(transactions_train):,}")
print(f"   - Test accounts: {len(accounts_test):,}")
print(f"   - Test transactions: {len(transactions_test):,}")

In [ ]:
print("💾 Saving split train/test tables...")

accounts_train.to_parquet("./data/two-table/accounts_train.parquet", index=False)
accounts_test.to_parquet("./data/two-table/accounts_test.parquet", index=False)
transactions_train.to_parquet("./data/two-table/transactions_train.parquet", index=False)
transactions_test.to_parquet("./data/two-table/transactions_test.parquet", index=False)

print("✅ All train/test splits saved to ./data/two-table/")

## 3. SDV (Synthetic Data Vault) Implementation

**About SDV:**
- Business Source License Python library for synthetic data generation
- Supports single-table and multi-table scenarios
- Uses statistical modeling and machine learning approaches
- Provides HMASynthesizer for hierarchical multi-table synthesis

**Key Features:**
- **Metadata Detection**: Automatically infers data types and relationships
- **Relationship Modeling**: Handles parent-child table relationships
- **Privacy Protection**: Generates synthetic data that preserves statistical properties while protecting individual privacy
- **Extensible**: Multiple synthesizer options (GaussianCopula, CTGAN, CopulaGAN, etc.)

**Limitations:**
- Current version only supports one parent per child table
- Complex multi-parent relationships require modeling simplification
- Performance scales with data complexity


In [ ]:
from sdv.metadata import Metadata
from sdv.multi_table import HMASynthesizer

print("🏗️ Building SDV metadata configuration...")

# Auto-detect metadata using only the relevant tables
metadata = Metadata.detect_from_dataframes(
    data={"account": accounts_train, "transaction": transactions_train}, infer_keys="primary_and_foreign"
)

print("✅ Base metadata auto-detected with relationships")

# View auto-detected relationships graphically
metadata.visualize()

# Inspect relationships and table configuration as raw dictionary
metadata_dict = metadata.to_dict()
print("\n📋 Complete SDV Metadata Dictionary:")
print(metadata_dict)

### 3.2 SDV Model Training

**HMASynthesizer Overview:**
- **Hierarchical Modeling**: Learns parent-child relationships
- **Statistical Approach**: Uses copulas and Gaussian distributions
- **Multi-step Process**: 
  1. Preprocesses tables and infers constraints
  2. Learns relationships between parent and child tables
  3. Models individual table distributions
  
**Training Phases:**
- **Preprocess Tables**: Data cleaning and type inference
- **Learning Relationships**: Analyzing foreign key dependencies  
- **Modeling Tables**: Learning statistical distributions for synthesis


In [ ]:
import time

print("🚀 Starting SDV training process...")
print("This will involve multiple phases - preprocessing, relationship learning, and table modeling")

start_time = time.time()

# Initialize the HMASynthesizer with our configured metadata
print("🔧 Initializing HMASynthesizer...")
synthesizer = HMASynthesizer(metadata)

# Fit the synthesizer on training data
# This process will:
# 1. Preprocess all tables (clean data, infer constraints)
# 2. Learn multi-table relationships
# 3. Model the statistical distributions of each table
print("📊 Training synthesizer on multi-table data...")
synthesizer.fit({"account": accounts_train, "transaction": transactions_train})

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print("✅ SDV training completed successfully!")
print(f"⏱️ Total training time: {elapsed_minutes:.2f} minutes")

# Report table sizes for clarity
print("📈 Training data breakdown:")
print(f"   - Accounts: {len(accounts_train):,}")
print(f"   - Transactions: {len(transactions_train):,}")

### 3.3 SDV Synthetic Data Generation

**Generation Process:**
- **Scale Parameter**: Controls the number of synthetic records (1.25 = same size as training data)
- **Hierarchical Generation**: First generates parent records (accounts), then child records (transactions)
- **Relationship Preservation**: Ensures all synthetic transfers reference valid synthetic customers
- **Statistical Sampling**: Uses learned distributions to create realistic synthetic data


In [ ]:
print("🎲 Starting SDV synthetic data generation...")
print("Generating synthetic data using learned statistical distributions...")

start_time = time.time()

# Generate synthetic data with the same number of records as training data
print("⚙️ Generating 1.25x the training data size...")
sdv_synthetic_data = synthesizer.sample(scale=1.25)

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print("✅ SDV generation completed successfully!")
print(f"⏱️  Generation time: {elapsed_minutes:.2f} minutes")

# Calculate synthetic record counts
synthetic_account_count = len(sdv_synthetic_data["account"])
synthetic_transaction_count = len(sdv_synthetic_data["transaction"])

total_synthetic_records = synthetic_account_count + synthetic_transaction_count
generation_rate = total_synthetic_records / (end_time - start_time)

print(f"🚀 Generation rate: {generation_rate:,.0f} records/second")
print("📊 Synthetic data breakdown:")
print(f"   - Accounts: {synthetic_account_count:,}")
print(f"   - Transactions: {synthetic_transaction_count:,}")

# Preview of generated synthetic data
sdv_synthetic_data["account"].head()
sdv_synthetic_data["transaction"].head()

In [ ]:
# Save SDV synthetic data for comparison
output_folder = "./data/two-table/"

synthetic_files = {
    "account": f"{output_folder}sdv_account.parquet",
    "transaction": f"{output_folder}sdv_transaction.parquet",
}

for table_name, file_path in synthetic_files.items():
    sdv_synthetic_data[table_name].to_parquet(file_path, index=False)
    print(f"💾 Saved {table_name} synthetic data to: {file_path}")

## 4. MOSTLY AI Implementation

**About MOSTLY AI Synthetic Data SDK:**
- Open-source (Apache 2) synthetic data SDK with advanced AI capabilities
- Also cloud-based service with enterprise-grade security and compliance
- Supports complex multi-table scenarios with multiple foreign keys
- Uses deep learning and autoregressive-based models

**Getting Started:**
- **API Access**: Requires valid API credentials for cloud platform access
- **API Key Generation**: Get your free API key at: https://app.mostly.ai/settings/api-keys

**Key Advantages:**
- **Advanced AI Models**: Utilizes state-of-the-art generative AI including language models
- **Mixed Data Types**: Excels at both tabular and text data synthesis
- **Enterprise Features**: Privacy guarantees, compliance reporting, and scalability

**Architecture:**
- **Tabular Models**: For structured data (demographics, financials)
- **Language Models**: For text fields (names, addresses, emails) using LLMs like Llama-3.2
- **Sequential Models**: For time-series and ordered data patterns

In [ ]:
from mostlyai.sdk import MostlyAI

print("🔧 Initializing MOSTLY AI Synthetic Data SDK...")

# Initialize MOSTLY AI Synthetic Data SDK
mostly = MostlyAI(local=True)


print("✅ MOSTLY AI Synthetic Data SDK initialized successfully")

### 4.1 MOSTLY AI Configuration Summary

**Berka Two-Table Setup Highlights:**

- **Accounts Table**
  - Primary Key: `account_id`
  - Referenced by transactions table as foreign key

- **Transactions Table**
  - Foreign Key: `account_id` (context)
  - Context-aware modeling for financial records
  

In [ ]:
print("⚙️ Configuring streamlined MOSTLY AI generator...")
print("Setting up minimal two-table configuration for account and transaction data...")

# Configure the generator for the reduced two-table setup
config = {
    "name": "Berka Accounts & Transactions Generator",
    "tables": [
        {
            "name": "account",
            "data": accounts_train,
            "primary_key": "account_id",
            "tabular_model_configuration": {"enable_model_report": False},
        },
        {
            "name": "transaction",
            "data": transactions_train,
            "primary_key": "trans_id",
            "foreign_keys": [{"column": "account_id", "referenced_table": "account", "is_context": True}],
            "tabular_model_configuration": {"enable_model_report": False},
        },
    ],
}

print("✅ MOSTLY AI generator configuration ready.")

### 4.2 MOSTLY AI Training Process

**Training Process:**
1. **Upload Data**: Send training data securely to MOSTLY AI cloud
2. **Model Configuration**: Apply the complex multi-table configuration
3. **AI Training**: Use advanced generative models including LLMs
4. **Quality Validation**: Automatic quality checks during training


In [ ]:
print("🚀 Starting MOSTLY AI training...")
print("📤 Uploading training data to secure MOSTLY AI cloud platform...")

start_time = time.time()

# Train the MOSTLY AI generator with our advanced configuration
# This will:
# 1. Upload training data securely to the cloud
# 2. Configure both tabular and language models
# 3. Train AI models for each table and their relationships
# 4. Wait for training completion with progress monitoring
g = mostly.train(config=config, start=True, wait=True)

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

print("✅ MOSTLY AI training completed successfully!")
print(f"⏱️ Total training time: {elapsed_minutes:.2f} minutes")
print("🧠 Advanced AI models trained for multi-table synthesis")

In [ ]:
print("🎲 Starting MOSTLY AI synthetic data generation...")
print("🌩️ Using cloud-based AI models for high-quality multi-table synthesis...")

start_time = time.time()

# Generate synthetic data using the trained MOSTLY AI generator
# Key advantages over SDV:
# - Handles foreign keys properly
# - Maintains statistical relationships across tables

print(f"📊 Generating {len(accounts_train):,} synthetic account records...")

sd = mostly.generate(g, size=len(accounts_train))
mostlyai_synthetic_data = sd.data()

end_time = time.time()
elapsed_minutes = (end_time - start_time) / 60

# Calculate generation statistics
total_records = sum(len(mostlyai_synthetic_data[table]) for table in mostlyai_synthetic_data.keys())
generation_rate = total_records / (end_time - start_time)

print("✅ MOSTLY AI generation completed successfully!")
print(f"⏱️ Generation time: {elapsed_minutes:.2f} minutes")
print(f"🚀 Generation rate: {generation_rate:,.0f} records/second")

print("📊 Synthetic data breakdown:")
for table_name in mostlyai_synthetic_data:
    print(f"   - {table_name.capitalize()}: {len(mostlyai_synthetic_data[table_name]):,} rows")

# Preview
mostlyai_synthetic_data["account"].head()
mostlyai_synthetic_data["transaction"].head()

In [ ]:
# Save MOSTLY AI synthetic data for comparison
output_folder = "./data/two-table/"

mostlyai_files = {
    "account": f"{output_folder}mostlyai_account.parquet",
    "transaction": f"{output_folder}mostlyai_transaction.parquet",
}

for table_name, file_path in mostlyai_files.items():
    mostlyai_synthetic_data[table_name].to_parquet(file_path, index=False)
    print(f"💾 Saved {table_name} synthetic data to: {file_path}")

## 5. Synthetic Data Quality Assessment

After generating synthetic data using both SDV and MOSTLY AI, it's crucial to comprehensively evaluate the quality, privacy, and integrity of the generated datasets. This section provides a multi-faceted quality assessment framework that ensures our synthetic data meets production standards.

## 5.1 Statistical Quality Assessment with MOSTLY AI QA Library

The MOSTLY AI QA library provides enterprise-grade quality assessment capabilities that evaluate synthetic data across multiple dimensions. This assessment generates comprehensive HTML reports and quantitative metrics that help understand:

- **Accuracy Scores**: Overall statistical fidelity of synthetic data
- **Distance to Closest Record (DCR)**: Privacy risk measurement 
- **Univariate & Bivariate Distributions**: Preservation of individual column and column-pair statistics
- **Correlation Analysis**: Maintenance of relationships between variables
- **Similarity Metrics**: Overall resemblance to training data while avoiding overfitting

In [ ]:
# Import and initialize the quality assessment framework
from mostlyai import qa

# Initialize logging to see detailed evaluation progress
qa.init_logging()
print("🔍 Quality assessment framework initialized")

In [ ]:
# Load the split files from the disk
transactions_train = pd.read_parquet("./data/two-table/transactions_train.parquet")
transactions_test = pd.read_parquet("./data/two-table/transactions_test.parquet")
accounts_train = pd.read_parquet("./data/two-table/accounts_train.parquet")
accounts_test = pd.read_parquet("./data/two-table/accounts_test.parquet")

print("📂 Training and test datasets loaded successfully")

In [ ]:
print("📊 Evaluating SDV Transaction synthetic data quality...")

# Load the SDV synthetic dataset
sdv_transaction = pd.read_parquet("./data/two-table/sdv_transaction.parquet")

# Define ID columns to exclude from QA analysis (but keep account_id for context linkage)
id_columns_to_exclude = ["trans_id"]


def remove_id_columns(df, columns_to_remove):
    return df.drop(columns=[col for col in columns_to_remove if col in df.columns])


# Prepare data for QA
sdv_transaction_qa = remove_id_columns(sdv_transaction, id_columns_to_exclude)
transactions_train_qa = remove_id_columns(transactions_train, id_columns_to_exclude)
transactions_test_qa = remove_id_columns(transactions_test, id_columns_to_exclude)

report_path, metrics = qa.report(
    syn_tgt_data=sdv_transaction_qa,
    trn_tgt_data=transactions_train_qa,
    hol_tgt_data=transactions_test_qa,
    syn_ctx_data=pd.read_parquet("./data/two-table/sdv_account.parquet"),
    trn_ctx_data=accounts_train,
    hol_ctx_data=accounts_test,
    ctx_primary_key="account_id",
    tgt_context_key="account_id",
    max_sample_size_embeddings=10_000,
    report_path="sdv_transaction_qa_report.html",
)

print(f"📋 SDV Transaction Quality Report saved to: {report_path}")
print("\n📈 SDV Transaction Quality Metrics:")
print(metrics.model_dump_json(indent=4))

# Print summary scores
sdv_transaction_accuracy = metrics.accuracy.overall
sdv_transaction_dcr_share = metrics.distances.dcr_share
print("\n🎯 SDV Transaction Summary:")
print(f"   Overall Accuracy: {sdv_transaction_accuracy:.3f}")
print(f"   DCR Share: {sdv_transaction_dcr_share:.3f}")

In [ ]:
print("📊 Evaluating MOSTLY AI Transactions synthetic data quality...")

# Load the MOSTLY AI synthetic dataset
mostlyai_transaction = pd.read_parquet("./data/two-table/mostlyai_transaction.parquet")

# Define ID columns to exclude from QA analysis (keep account_id)
id_columns_to_exclude = ["trans_id"]


def remove_id_columns(df, columns_to_remove):
    return df.drop(columns=[col for col in columns_to_remove if col in df.columns])


# Prepare transaction data
mostlyai_transaction_qa = remove_id_columns(mostlyai_transaction, id_columns_to_exclude)
transactions_train_qa = remove_id_columns(transactions_train, id_columns_to_exclude)
transactions_test_qa = remove_id_columns(transactions_test, id_columns_to_exclude)

report_path, metrics = qa.report(
    syn_tgt_data=mostlyai_transaction_qa,
    trn_tgt_data=transactions_train_qa,
    hol_tgt_data=transactions_test_qa,
    syn_ctx_data=pd.read_parquet("./data/two-table/mostlyai_account.parquet"),
    trn_ctx_data=accounts_train,
    hol_ctx_data=accounts_test,
    ctx_primary_key="account_id",
    tgt_context_key="account_id",
    max_sample_size_embeddings=10_000,
    report_path="mostlyai_transaction_qa_report.html",
)

print(f"📋 MOSTLY AI Transaction Quality Report saved to: {report_path}")
print("\n📈 MOSTLY AI Transaction Quality Metrics:")
print(metrics.model_dump_json(indent=4))

# Print summary scores
mostlyai_transaction_accuracy = metrics.accuracy.overall
mostlyai_transaction_dcr_share = metrics.distances.dcr_share
print("\n🎯 MOSTLY AI Transaction Summary:")
print(f"   Overall Accuracy: {mostlyai_transaction_accuracy:.3f}")
print(f"   DCR Share: {mostlyai_transaction_dcr_share:.3f}")

In [ ]:
# Add a final comparison section
print("\n" + "=" * 60)
print("🏆 FINAL COMPARISON")
print("=" * 60)
print(f"SDV Transaction      - Accuracy: {sdv_transaction_accuracy:.3f}, DCR Share: {sdv_transaction_dcr_share:.3f}")
print(
    f"MOSTLY AI Transaction- Accuracy: {mostlyai_transaction_accuracy:.3f}, DCR Share: {mostlyai_transaction_dcr_share:.3f}"
)

print("\n🔍 METRIC INTERPRETATION:")
print("• Higher accuracy = better statistical fidelity")
print("• DCR Share ~0.5 = optimal privacy-utility balance")

print("\n📊 ANALYSIS:")
print("• MOSTLY AI consistently shows higher accuracy than SDV")
print("• Both frameworks maintain reasonable DCR Share values around 0.5")
print("• MOSTLY AI handles multi-table relationships and foreign keys with greater precision")

print("\n⚠️  RECOMMENDATION:")
print("• Review detailed HTML reports for nuanced privacy insights")
print("• Pay attention to discriminator AUC and feature-wise similarity scores")
print("• Align final choice with your privacy-utility balance requirements")